### Import and install

In [14]:
import nltk
import string
import pandas as pd
import numpy as np
import copy
nltk.download('words')

[nltk_data] Downloading package words to /Users/jackcook/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

### Get 5 letter words

In [15]:
words_5l = sorted(list(set([w.upper() for w in nltk.corpus.words.words() if len(w) == 5])))
print(len(words_5l))
words_5l[:10]

9972


['AALII',
 'AARON',
 'ABACA',
 'ABACK',
 'ABAFF',
 'ABAFT',
 'ABAMA',
 'ABASE',
 'ABASH',
 'ABASK']

Arrange as a DataFrame

In [16]:
arr_words_5l = np.array([list(w) for w in words_5l])
df_words_5l = pd.DataFrame(data=arr_words_5l,
                           columns=[f'letter_{i+1}' for i in range(5)])

df_words_5l.head()

,letter_1,letter_2,letter_3,letter_4,letter_5
0,A,A,L,I,I
1,A,A,R,O,N
2,A,B,A,C,A
3,A,B,A,C,K
4,A,B,A,F,F


In [93]:
df_words_5l.columns

Index(['letter_1', 'letter_2', 'letter_3', 'letter_4', 'letter_5'], dtype='object')

Get list of letters at each position, ordered by their frequency

In [59]:
dict_most_likely[1]

array(['S', 'A', 'C', 'B', 'T', 'P', 'M', 'D', 'G', 'R', 'F', 'L', 'H',
       'W', 'K', 'U', 'N', 'E', 'O', 'I', 'V', 'J', 'Y', 'Z', 'Q', 'X'],
      dtype=object)

Initialise game

In [60]:
Counter(['a', 'a', 'b'])

Counter({'a': 2, 'b': 1})

In [81]:
MyGame.dict_most_likely_letters

defaultdict(str,
            {1: array(['S', 'A', 'C', 'B', 'T', 'P', 'M', 'D', 'G', 'R', 'F', 'L', 'H',
                    'W', 'K', 'U', 'N', 'E', 'O', 'I', 'V', 'J', 'Y', 'Z', 'Q', 'X'],
                   dtype=object),
             2: array(['A', 'O', 'E', 'I', 'U', 'R', 'L', 'H', 'N', 'T', 'P', 'C', 'Y',
                    'M', 'W', 'S', 'D', 'B', 'K', 'G', 'V', 'X', 'Z', 'F', 'Q', 'J'],
                   dtype=object),
             3: array(['A', 'R', 'I', 'O', 'N', 'E', 'L', 'U', 'T', 'S', 'M', 'D', 'B',
                    'G', 'C', 'P', 'K', 'V', 'W', 'Y', 'H', 'F', 'Z', 'X', 'J', 'Q'],
                   dtype=object),
             4: array(['E', 'A', 'I', 'N', 'R', 'O', 'L', 'T', 'S', 'U', 'C', 'G', 'D',
                    'M', 'K', 'P', 'B', 'H', 'V', 'F', 'Z', 'Y', 'W', 'J', 'X', 'Q'],
                   dtype=object),
             5: array(['E', 'Y', 'A', 'T', 'R', 'N', 'L', 'S', 'D', 'H', 'O', 'I', 'K',
                    'M', 'C', 'P', 'G', 'U', 'X', 'F', 'B', 'W'

In [90]:
MyGame.dict_most_likely[1]

Counter({'A': 33,
         'B': 13,
         'C': 28,
         'D': 7,
         'F': 20,
         'H': 3,
         'I': 12,
         'K': 7,
         'L': 3,
         'N': 3,
         'O': 14,
         'P': 15,
         'Q': 7,
         'R': 4,
         'S': 55,
         'U': 9,
         'V': 1,
         'W': 11,
         'Y': 1,
         'Z': 1})

In [ ]:
test_df

In [95]:
test_df = pd.DataFrame([['A','B'],['C','D']], columns=['a','b'])
test_df.agg(''.join, axis=1)

0    AB
1    CD
dtype: object

In [88]:
from collections import defaultdict, Counter


# All letters in alphabet
alpha_list = list(string.ascii_uppercase)

class Game:
    
    def __init__(self, df_all_5l_words, list_possible_letters, dict_most_likely_letters):
        
        self.possible_letters = copy.deepcopy(alpha_list)
        self.dict_misplaced_letters = Counter()
        self.df_possible_5l_words = df_all_5l_words.copy(deep=True)
        
        self.dict_letters = defaultdict(str)
        for i in range(5):
            self.dict_letters[i+1] = None
        
        self.dict_most_likely = defaultdict(str)
        for i in range(5):
            self.dict_most_likely[i+1] = Counter(df_words_5l[f'letter_{i+1}'])
        

    def calculate_score(self, letters):
        '''
        Based on a set of letters, sum their frequency score at each position
        '''
        
        score = 0
        for i, l in enumerate(list(letters.upper())):
            score += self.dict_most_likely[i+1][l]
            
        return score
    
    vect_calculate_score = np.vectorize(calculate_score)
        
    
    def guess(self):
        '''
        Given a game state, produces the best guess possible based on the 'score' at
        each position.
        
        The list of possible words is already filtered for correct answers,
        and letters that fall within the list of possible list of letters for the remaining
        positions.
        '''
        
        self.df_possible_5l_words['word'] = self.df_possible_5l_words[['letter_1', 
                                                                      'letter_2', 
                                                                      'letter_3', 
                                                                      'letter_4', 
                                                                      'letter_5']].agg(''.join, axis=1)
        
        self.df_possible_5l_words['score'] = vect_calculate_score(self.df_possible_5l_words['word'])
        
    
    def update(self, guess, results):
        '''
        Takes a 5 letter guess as a string, and a list of results in the format:
        0 - incorrect
        1 - right letter, wrong place
        2 - right letter, right place
        
        Updates the game states (possible letters, misplaced letters, answers).
        
        Doesn't return anything
        '''
        
        assert len(guess) == 5, 'Guess must be 5 characters long'
        
        # Convert guess into list of letters
        list_guess = list(guess.upper())
        
        # Zip with results
        df_guess_results = pd.DataFrame(data=list(zip(list_guess, results)),
                                        columns=['letter', 'result'],
                                        index=np.arange(1,6))
        
        # To prevent iterating through already solved letters
        already_solved = [key for key, value in self.dict_letters.items() if value is not None]

        
        # Update correct answers
        df_corr_answers = df_guess_results.query('result==2')
        if df_corr_answers.shape[0] > 0:
            for idx, row in df_corr_answers.iterrows():
                
                # Prevent updates for previously solved letters
                if idx in already_solved:
                    pass
                else:
                    corr_letter = row['letter']
                    self.dict_letters[idx] = corr_letter
                
                    # If correct letter was previously guessed as a misplaced letter, remove it
                    if corr_letter in self.dict_misplaced_letters.keys():
                        self.dict_misplaced_letters[corr_letter] -= 1
                        
                    # And filter dataframe of possible words
                    self.df_possible_5l_words = self.df_possible_5l_words.query(f'letter_{idx}=="{corr_letter}"')
                    
                        
        # Add misplaced letters to our list, if it's a new letter
        df_mispl_answers = df_guess_results.query('result==1')
        if df_mispl_answers.shape[0] > 0:
            
            # Check how many we have of each letter that's misplaced
            guess_mispl_letters = df_mispl_answers['letter'].values
            dict_guess_mispl_letters = Counter(guess_mispl_letters)
            
            # Then update our dictionary of misplaced letters
            for key, value in dict_guess_mispl_letters.items():
                self.dict_misplaced_letters[key] = value       
                
        
        # Remove any incorrect letters from the list to guess from
        df_wrong_answers = df_guess_results.query('result==0')
        if df_wrong_answers.shape[0] > 0:
            
            for l in df_wrong_answers['letter'].unique():
                self.possible_letters.remove(l)
                
                
        # Finally, update list of possible 5 letter words by removing all rows where
        # for letters yet to be guessed, they don't fall in the list of possible letters
        yet_to_solve = [key for key, value in self.dict_letters.items() if value is None]
        for position in yet_to_solve:
            
            # Check all letters in a given position
            position_letters = self.df_possible_5l_words[f'letter_{position}']
            
            # Return a boolean list of whether that list is in the possible values or not
            position_in_possible_letters = [l in self.possible_letters for l in position_letters]
            
            # Filter
            self.df_possible_5l_words = self.df_possible_5l_words[position_in_possible_letters].copy(deep=True)
        
        
        # And recalibrate counter dictionary
        self.dict_most_likely = defaultdict(str)
        for i in range(5):
            self.dict_most_likely[i+1] = Counter(self.df_possible_5l_words[f'letter_{i+1}'])
    
        
MyGame = Game(df_words_5l, alpha_list, dict_most_likely)
MyGame.update('greet', [0, 1, 2, 0, 0])

In [90]:
MyGame.dict_most_likely[1]

Counter({'A': 33,
         'B': 13,
         'C': 28,
         'D': 7,
         'F': 20,
         'H': 3,
         'I': 12,
         'K': 7,
         'L': 3,
         'N': 3,
         'O': 14,
         'P': 15,
         'Q': 7,
         'R': 4,
         'S': 55,
         'U': 9,
         'V': 1,
         'W': 11,
         'Y': 1,
         'Z': 1})

In [57]:
mylist = ['a', 'b']
mylist.remove('b')
mylist

['a']